In [1]:
import espaloma as esp
import mbuild as mb
from openff.toolkit.topology import Molecule
from writers import foyer_xml_writer
from writers.foyer_xml_writer import mbuild_to_foyer_xml
from bondwalk import bond_walk
from bondwalk.bond_walk import MadAtom, MadBond, BondWalker
import ipywidgets as widgets
import os
import torch
import parmed as pmd
import networkx  as nx
from mbuild.lib.recipes.polymer import Polymer
if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/Users/madilyn/miniconda3/envs/ff_new2/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/Users/madilyn/miniconda3/envs/ff_new2/lib/python3.11/site-packages/mdtraj/geometry/order.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/Users/madilyn/miniconda3/envs/ff_new2/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/madilyn/miniconda3/envs/ff_new2/lib/python3.11/site-packages/lark/utils.py:163: DeprecationWarning:

# Parameterizing one molecule:

In [2]:
from mon_classes import PCPDTPT_HD

In [24]:
### set your molecule information

#mol = P3HT()
XML_FILEPATH = '/Users/madilyn/Projects/repos/forcefields/xml_files/y6.xml'
TYPED_FILEPATH = '/Users/madilyn/Projects/repos/forcefields/typed_mol2/y6.mol2'

In [3]:
def build_chain(monomer, length, min_energy):
    chain = Polymer()
    chain.add_monomer(compound=monomer,
                 indices=monomer.bond_indices,
                 separation=monomer.separation,
                 replace=monomer.replace,
                 orientation=monomer.orientations)
    chain.build(n=length)
    if min_energy == True:
        chain.energy_minimize()
    return chain

def mon_to_smiles(fragment):
    monomer = build_chain(fragment,1,min_energy=True)
    dimer = build_chain(fragment,2,min_energy=True)
    mon_dim = mb.Compound()
    mon_dim.add([monomer,dimer])
    monomer.translate([3,3,3])
    dimer.translate([-3,-3,-3])
    smiles_string = mon_dim.to_smiles()
    return mon_dim, smiles_string

In [4]:
pcpdt = PCPDTPT_HD()
smiles = pcpdt.to_smiles()

In [5]:
comp = Molecule.from_smiles(smiles,allow_undefined_stereo=True)
bonds = [b for b in comp.bonds]
for i in range(len(bonds)):
    bonds[i].bond_order = 1
b= BondWalker(comp)
molecule = b.fill_in_bonds()
molecule.visualize()

RadicalsNotSupportedError: The OpenFF Toolkit does not currently support parsing molecules with S- and P-block radicals. Found 1 radical electrons on molecule [H]C1=[C]SC2=C1[C@@](C([H])([H])[C@@]([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])(C([H])([H])[C@@]([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])C1=C2SC(c2c([H])n[c]c3=NSN=c23)=C1[H].

In [9]:
molecule_graph = esp.Graph(molecule)
espaloma_model = esp.get_model("latest")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph,charge_method="nn")

In [14]:
# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[0]
angle_forces = openmm_system.getForces()[2]
bond_forces = openmm_system.getForces()[3]
torsion_forces = openmm_system.getForces()[1]

In [17]:
type(bond_forces)

openmm.openmm.HarmonicBondForce

In [18]:


# get a parmed structure from openmm
topology = molecule.to_topology()
openmm_topology = topology.to_openmm()

structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)



for i in range(len(molecule.atoms)):
    if molecule.atoms[i].atomic_number == 6:
        molecule.atoms[i].name = 'C'
    if molecule.atoms[i].atomic_number == 1:
        molecule.atoms[i].name = 'H'
    if molecule.atoms[i].atomic_number == 7:
        molecule.atoms[i].name = 'N'
    if molecule.atoms[i].atomic_number == 16:
        molecule.atoms[i].name = 'S'
    if molecule.atoms[i].atomic_number == 8:
        molecule.atoms[i].name = 'O'
    if molecule.atoms[i].atomic_number == 9:
        molecule.atoms[i].name = 'F'

#generate type_map:
Gopenmm = nx.Graph()
Gparmed = nx.Graph()

#openmm:
for i in range(bond_forces.getNumBonds()):
    Gopenmm.add_edge(bond_forces.getBondParameters(index=i)[0],bond_forces.getBondParameters(index=i)[1])
#parmed
for b in structure.bonds:
    Gparmed.add_edge(b.atom1.idx,b.atom2.idx)

particle_types = []
type_map = dict()

#nx.rooted_tree_isomorphism
#in here we still need to check that one known index on one corresponds to the same index on the other....
tree_openmm = nx.bfs_tree(Gopenmm,0)
tree_parmed = nx.bfs_tree(Gparmed,0)
if nx.is_isomorphic(Gopenmm,Gparmed):
#if nx.isomorphism.tree_isomorphism(tree_openmm,tree_parmed):  <- want this work
    for i in range(pair_forces.getNumParticles()):
        pair_parms = pair_forces.getParticleParameters(index=i)
        sigma = pair_parms[1]/pair_parms[1].unit
        epsilon = pair_parms[2]/pair_parms[2].unit
        if (sigma, epsilon) not in particle_types:
            particle_types.append((sigma, epsilon))
        type_map[molecule.atoms[i].molecule_atom_index] = "".join([molecule.atoms[i].name ,
                                                                   str(particle_types.index((sigma, epsilon)))])


#generate dictionaries:

bond_types = []
bond_dict = dict()

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    l0 = bond_parms[2]/bond_parms[2].unit
    k = bond_parms[3]/bond_parms[3].unit
    bond_dict[type_map[bond_parms[0]],type_map[bond_parms[1]]] = {'k':k,'l0':l0}


angle_types = []
angle_dict = dict()

for i in range(angle_forces.getNumAngles()):
    angle_parms = angle_forces.getAngleParameters(index=i)
    k = angle_parms[4]/angle_parms[4].unit
    t0 = angle_parms[3]/angle_parms[3].unit
    angle_dict[type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]]] = {'k':k,'t0':t0}



dihedral_types = []
dihedral_dict = {}

for i in range(torsion_forces.getNumTorsions()):
    if i%6==0:
        periodicity=[]
        phase = []
        k = []
    dihedral_parms = torsion_forces.getTorsionParameters(index=i)
    periodicity.append(dihedral_parms[4])
    phase.append( dihedral_parms[5]/dihedral_parms[5].unit)
    k.append(dihedral_parms[6]/dihedral_parms[6].unit)
    dt = (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
                  type_map[dihedral_parms[3]])


    if periodicity[-1]==6:
        dihedral_dict[dt] = {'periodicity':periodicity,'k':k,'phase':phase}


nonbonded_types = []
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]#/nonbonded_parms[2].unit
    nonbonded_types.append((charge,sigma,epsilon))
    nonbonded_dict[(type_map[i])]={'charge':charge,'sigma':sigma,'epsilon':epsilon}

molecule.to_file('molecule.mol',file_format='mol')
os.system('obabel molecule.mol -O intermediate.mol2')
os.system('rm molecule.mol')

test = mb.load('intermediate.mol2')

for index in type_map:
    test[index].name = type_map[index]

os.system('rm intermediate.mol2')

test.energy_minimize()

test.visualize()

1 molecule converted


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [22]:
for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]#/nonbonded_parms[2].unit
    nonbonded_types.append((charge,sigma,epsilon))
    nonbonded_dict[(type_map[i])]={'charge':charge,'sigma':sigma,'epsilon':epsilon}
nonbonded_dict

{'C0': {'charge': -0.09526515007019043,
  'sigma': 0.3399669508423535,
  'epsilon': Quantity(value=0.4577296, unit=kilojoule/mole)},
 'C1': {'charge': 0.007520198822021484,
  'sigma': 0.3399669508423535,
  'epsilon': Quantity(value=0.359824, unit=kilojoule/mole)},
 'S2': {'charge': 0.10220050811767578,
  'sigma': 0.35635948725613575,
  'epsilon': Quantity(value=1.046, unit=kilojoule/mole)},
 'N3': {'charge': -0.3814876079559326,
  'sigma': 0.3249998523775958,
  'epsilon': Quantity(value=0.7112800000000001, unit=kilojoule/mole)},
 'O4': {'charge': -0.49110841751098633,
  'sigma': 0.2959921901149463,
  'epsilon': Quantity(value=0.87864, unit=kilojoule/mole)},
 'F5': {'charge': -0.11919379234313965,
  'sigma': 0.3118145513491188,
  'epsilon': Quantity(value=0.255224, unit=kilojoule/mole)},
 'C6': {'charge': 0.21631240844726562,
  'sigma': 0.3399669508423535,
  'epsilon': Quantity(value=0.87864, unit=kilojoule/mole)},
 'H7': {'charge': 0.03691279888153076,
  'sigma': 0.2649532787749369,
  

In [25]:
mbuild_to_foyer_xml(
    file_name=XML_FILEPATH,
    compound=test,
    bond_params=bond_dict,
    angle_params=angle_dict,
    dihedral_params=dihedral_dict,
    dihedral_type="periodic",
    non_bonded_params=nonbonded_dict,
    combining_rule="geometric",
    name="",
    version="",
    coulomb14scale=1.0,
    lj14scale=1.0)

test.save(TYPED_FILEPATH,overwrite=True)

In [26]:
test = mb.load("typed_mol2/y6.mol2")
test.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol